In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM , Embedding,SpatialDropout1D
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
df=pd.read_csv('/home/user/Downloads/Tweets.csv')

In [3]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [4]:
df.isnull().sum()

tweet_id                            0
airline_sentiment                   0
airline_sentiment_confidence        0
negativereason                   5462
negativereason_confidence        4118
airline                             0
airline_sentiment_gold          14600
name                                0
negativereason_gold             14608
retweet_count                       0
text                                0
tweet_coord                     13621
tweet_created                       0
tweet_location                   4733
user_timezone                    4820
dtype: int64

In [5]:
#creating a new dataframe with only 2 columns the review and the respected sentiment
data=df[['airline_sentiment','text']]

In [6]:
data.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [7]:
data['airline_sentiment'].unique()

array(['neutral', 'positive', 'negative'], dtype=object)

In [8]:
dataset = data[data.airline_sentiment != 'neutral']

In [9]:
Stopwords=set(stopwords.words('english'))

In [10]:
dataset['text']=dataset['text'].apply(lambda x:x.lower())

<ipython-input-10-722df6e9d014>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['text']=dataset['text'].apply(lambda x:x.lower())


In [11]:
dataset.text[17]

"@virginamerica  i flew from nyc to sfo last week and couldn't fully sit in my seat due to two large gentleman on either side of me. help!"

In [12]:
dataset.text=dataset['text'].apply(lambda x: ' '.join(word for word in x.split() if word not in Stopwords))

/home/user/.local/lib/python3.8/site-packages/pandas/core/generic.py:5159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [13]:
dataset.text[17]

'@virginamerica flew nyc sfo last week fully sit seat due two large gentleman either side me. help!'

In [14]:
dataset.text=dataset['text'].apply(lambda x: re.sub('[^a-zA-Z\s]','',x))

In [15]:
dataset.text[17]

'virginamerica flew nyc sfo last week fully sit seat due two large gentleman either side me help'

In [16]:
tokenizer=Tokenizer(num_words=5000)
tokenizer.fit_on_texts(dataset.text.values)

In [17]:
dataset.head()

,airline_sentiment,text
1,positive,virginamerica plus added commercials experienc...
3,negative,virginamerica really aggressive blast obnoxiou...
4,negative,virginamerica really big bad thing
5,negative,virginamerica seriously would pay flight seat...
6,positive,virginamerica yes nearly every time fly vx ear...


In [18]:
seq = tokenizer.texts_to_sequences(dataset.text.values)

In [19]:
seq

[[42, 442, 1139, 2187, 100],
 [42, 57, 2950, 3703, 3704, 823, 2951, 3705, 19, 369, 2188],
 [42, 57, 331, 117, 372],
 [42, 283, 33, 183, 2, 122, 1959, 57, 117, 372, 78, 1646],
 [42, 107, 1197, 202, 13, 50, 1527, 3706, 927, 76, 359],
 [42, 116, 280, 1001],
 [42, 230, 415, 28, 252, 66, 53],
 [42, 968, 525, 95, 121, 1001],
 [42, 45, 535, 137, 1436, 472, 125, 19, 60, 746, 247, 125, 162, 1273],
 [42, 15, 78, 1647, 1349, 99, 54, 84, 1528, 359, 118],
 [42, 9],
 [42, 858, 22, 747],
 [42, 768, 91, 1198, 710, 2, 238, 684, 46, 434, 143, 45, 380, 928, 1274],
 [42,
  503,
  404,
  290,
  61,
  215,
  1960,
  350,
  80,
  87,
  109,
  1529,
  3707,
  584,
  1530,
  53,
  12],
 [78, 42],
 [42, 67, 33, 3708, 180, 38, 86, 50, 32],
 [42, 91, 1437, 451, 320, 133, 1438, 122, 260, 889],
 [42, 103],
 [42, 103, 2952, 427, 66, 1439],
 [42, 44, 1140, 929, 1961, 929, 637, 44, 321, 80, 359, 86, 209, 1076],
 [42, 295, 1077, 1078, 1648, 320, 313, 165, 1079, 104],
 [42,
  452,
  304,
  2953,
  332,
  443,
  272,
  2

In [20]:
seq=pad_sequences(seq)
seq.shape

(11541, 22)

In [21]:
y = pd.get_dummies(dataset['airline_sentiment'],drop_first=True).values

In [22]:
x_train,x_test,y_train,y_test = train_test_split(seq, y, test_size = 0.2,random_state = 0)

In [23]:
model = Sequential()
model.add(Embedding(5000,128,input_length=seq.shape[1])) 
model.add(SpatialDropout1D(.4))
model.add(LSTM(256, dropout=0.5, recurrent_dropout= 0.5)) 
model.add(Dense(1,activation = 'sigmoid'))

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 22, 128)           640000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 22, 128)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 256)               394240    
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 1,034,497
Trainable params: 1,034,497
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [26]:
model.fit(x_train,y_train,batch_size=100,epochs=10)

Epoch 1/10
93/93 [==============================] - 8s 81ms/step - loss: 0.4425 - accuracy: 0.8190
Epoch 2/10
93/93 [==============================] - 7s 78ms/step - loss: 0.2144 - accuracy: 0.9135
Epoch 3/10
93/93 [==============================] - 7s 79ms/step - loss: 0.1617 - accuracy: 0.9378
Epoch 4/10
93/93 [==============================] - 7s 79ms/step - loss: 0.1281 - accuracy: 0.9504
Epoch 5/10
93/93 [==============================] - 7s 79ms/step - loss: 0.1073 - accuracy: 0.9599
Epoch 6/10
93/93 [==============================] - 7s 78ms/step - loss: 0.0930 - accuracy: 0.9658
Epoch 7/10
93/93 [==============================] - 7s 78ms/step - loss: 0.0754 - accuracy: 0.9705
Epoch 8/10
93/93 [==============================] - 7s 78ms/step - loss: 0.0700 - accuracy: 0.9741
Epoch 9/10
93/93 [==============================] - 7s 78ms/step - loss: 0.0590 - accuracy: 0.9774
Epoch 10/10
93/93 [==============================] - 7s 78ms/step - loss: 0.0518 - accuracy: 0.9799


In [27]:
score=model.evaluate(x_test,y_test)

73/73 [==============================] - 1s 9ms/step - loss: 0.3221 - accuracy: 0.9186


In [28]:
tweets = ["virginamerica has the most incredible customer service i've ever experienced! so refreshing!"]

In [29]:
tokenizer.fit_on_texts(tweets)

In [30]:
filtered_twts  =[]
for word in tweets:
  if word not in Stopwords:
    filtered_twts.append(word)

In [31]:
filtered_twts

["virginamerica has the most incredible customer service i've ever experienced! so refreshing!"]

In [32]:
seq1 = tokenizer.texts_to_sequences(filtered_twts)

In [33]:
seq1 = pad_sequences(seq1, maxlen=22)

In [34]:
y_pred = model.predict_classes(seq1)
y_pred

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[0]], dtype=int32)